In [1]:
import sys
sys.path.append("./utils/")
import random
from data_path import *
from glob import glob
from os.path import join, basename, dirname, samefile
from tqdm import tqdm
from skimage import data, color, exposure 
from skimage.io import imread
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score,f1_score
import re
%load_ext autoreload
%autoreload 1
%aimport data_path

In [73]:
X = list()
Y = list()
files = get_all_files(cropped=True)
for file in tqdm(files):
    image = imread(file)
    image = color.rgb2hsv(image) # представляем изображение как трехмерный массив [x,y,3], вместо 3 слоев rgb - hue, saturation и v
    #используем только hue, так как в соло оно показательно
    fd = exposure.histogram(image[:,:,0])[0]
    X.append(fd)
    Y.append(get_genre_id(file))
X = np.asarray(X)
Y = np.asarray(Y)
np.save(join(FEATURES["RF"],'X_hist'),X)
np.save(join(FEATURES["RF"],'Y'),Y)

100%|█████████████████████████████████████████████████████████████████████████| 334363/334363 [47:50<00:00, 116.50it/s]


In [2]:
files_cropped = get_all_files(cropped=True)
files_origin = get_all_files(cropped=False)

In [6]:
X = np.load(join(FEATURES["RF"],'X_hist.npy'))
Y = np.load(join(FEATURES["RF"],'Y.npy'))

In [7]:
classifier = RandomForestClassifier(n_estimators=50)
cross_val_score(classifier, X, Y)

array([ 0.22244054,  0.22578417,  0.2228573 ])

In [74]:
uncut_files = {(dirname(f),re.sub("[0-9]+_[0-9]+_","",basename(f))) for f in files_cropped}

In [75]:
RF_train_files = set(random.sample(uncut_files,9000))
RF_test_files = uncut_files.difference(RF_train_files)

In [3]:
files_cropped_dict = dict()
for i,f in tqdm(enumerate(files_cropped)):
    directory = dirname(f)
    name_origin = re.sub("[0-9]+_[0-9]+_","",basename(f))
    try:
        files_cropped_dict[(directory, name_origin)].append(i)
    except KeyError:
        files_cropped_dict[(directory, name_origin)] = [i]

334363it [00:04, 81379.11it/s]


In [80]:
RF_train_indices = list()
for f in RF_train_files:
    RF_train_indices.extend(files_cropped_dict[f])
RF_train_indices = np.asarray(RF_train_indices)

In [88]:
X_train_RF = X[RF_train_indices]
Y_train_RF = Y[RF_train_indices]

In [101]:
mask = np.ones((X.shape[0])).astype(bool)
mask[RF_train_indices] = False
X_test_RF = X[mask]
Y_test_RF = Y[mask]

In [102]:
classifier = RandomForestClassifier()
classifier.fit(X_train_RF,Y_train_RF)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [153]:
X_RF_NN = list()
Y_RF_NN = list()
for f in tqdm(RF_test_files):
    indices = files_cropped_dict[f]
    ys = classifier.predict(X[indices])
    ans = np.zeros((150, 18)).astype(int)
    i = 0
    for y in ys:
        ans[i][y-1] = 1
        i += 1
        if i >= 150:
            break
    X_RF_NN.append(ans.flatten())
    Y_RF_NN.append(Y[indices][0])
X_RF_NN = np.asarray(X_RF_NN)
Y_RF_NN = np.asarray(Y_RF_NN)

100%|████████████████████████████████████████████████████████████████████████████| 9037/9037 [00:08<00:00, 1038.75it/s]


In [156]:
np.save("X_RF_NN",)

(9037,)